<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/Instant_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ee.Authenticate()
ee.Initialize(project='ee-p66121061')
map = geemap.Map()
map

In [ ]:
# Compute Trend for the whole time series
trend_mndwi = mndwi_collection.select(['system:time_start','mndwi_aoi']).reduce(ee.Reducer.linearFit())
trend_ndvi = ndvi_collection.select(['system:time_start','ndvi_aoi']).reduce(ee.Reducer.linearFit())
trend_ndbi = ndbi_collection.select(['system:time_start','ndbi_aoi']).reduce(ee.Reducer.linearFit())

#MNDWI
# 提取斜率
slope_mndwi = trend_mndwi.select('scale')

# 計算斜率的最小值和最大值
min_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_mndwi = slope_mndwi.subtract(min_slope_mndwi).divide(max_slope_mndwi.subtract(min_slope_mndwi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_mndwi = trend_mndwi.addBands(normalized_slope_mndwi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_mndwi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6']}, 'Normalized Trend_MNDWI')

In [ ]:
#NDVI
# 提取斜率
slope_ndvi = trend_ndvi.select('scale')

# 計算斜率的最小值和最大值
min_slope_ndvi = ee.Number(slope_ndvi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_ndvi = ee.Number(slope_ndvi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_ndvi = slope_ndvi.subtract(min_slope_ndvi).divide(max_slope_ndvi.subtract(min_slope_ndvi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_ndvi = trend_ndvi.addBands(normalized_slope_ndvi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_ndvi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#d7191c','#fdae61','#ffffbf','#99B718','#66A000']}, 'Normalized Trend_NDVI')

In [ ]:
#NDBI
# 提取斜率
slope_ndbi = trend_ndbi.select('scale')

# 計算斜率的最小值和最大值
min_slope_ndbi = ee.Number(slope_ndbi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_ndbi = ee.Number(slope_ndbi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_ndbi = slope_ndbi.subtract(min_slope_ndbi).divide(max_slope_ndbi.subtract(min_slope_ndbi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_ndbi = trend_ndbi.addBands(normalized_slope_ndbi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_ndbi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#66A000','#99B718','#ffffbf','#fdae61','#d7191c']}, 'Normalized Trend_NDBI')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# 設定檔案路徑與參數
image_folder = 'your_image_folder'  # 放置影像的資料夾
start_year = 2000
end_year = 2019
change_threshold = 30  # 設定變化閾值

def process_image(image_path):
    # 讀取影像並轉為灰度圖
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def detect_changes(img1, img2, threshold):
    # 計算相鄰年份的圖像差異
    diff = cv2.absdiff(img1, img2)
    # 閾值處理：超過threshold的區域設為白色，其他區域設為黑色
    _, binary_diff = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    return binary_diff

# 逐年處理影像
for year in range(start_year, end_year):
    img1_path = os.path.join(image_folder, f'year_{year}.jpg')
    img2_path = os.path.join(image_folder, f'year_{year + 1}.jpg')

    # 讀取相鄰兩年的影像
    img1 = process_image(img1_path)
    img2 = process_image(img2_path)

    # 檢測變化
    binary_diff = detect_changes(img1, img2, change_threshold)

    # 顯示並儲存結果
    plt.imshow(binary_diff, cmap='gray')
    plt.title(f'Changes from {year} to {year + 1}')
    plt.axis('off')
    plt.show()

    # 儲存變化的二值化影像
    output_path = os.path.join(image_folder, f'change_{year}_{year+1}.jpg')
    cv2.imwrite(output_path, binary_diff)